In [444]:
import pandas as pd # type: ignore
import numpy as np # type: ignore
import re # type: ignore
import string # type: ignore
import nltk # type: ignore
from nltk.corpus import stopwords # type: ignore
from nltk.tokenize import word_tokenize # type: ignore
from nltk.stem import PorterStemmer # type: ignore
from textblob import Word # type: ignore
from autocorrect import Speller # type: ignore
import matplotlib.pyplot as plt # type: ignore

In [445]:
df = pd.read_csv('../../data/Tweets.csv')
data = df[['text','airline_sentiment']]

In [ ]:
data.head()

In [ ]:
data.info()

### __Data Cleaning__
1. Missing Values
2. Data Types
3. Duplicates

In [ ]:
# checking for missing values
data.isnull().sum()

In [ ]:
# checking the description of the data
data.describe()

In [ ]:
# ensuring that the "text" and "airline_sentiment" columns has unique datatypes
num_text_types = data['text'].apply(type).nunique()
num_sentiment_types = data['airline_sentiment'].apply(type).nunique()
print(f"n of datatypes in 'text': {num_text_types}")
print(f"n of datatypes in 'airline_sentiment': {num_sentiment_types}")

In [ ]:
# gettting the row value of the duplicated rows in text column
duplicate_count = data['text'].duplicated().sum()
print(f"n of duplicate rows in 'text': {duplicate_count}")

In [452]:
# dropping the duplicated rows in "text" column
data = data.drop_duplicates(subset=['text'])

In [ ]:
# checking the description of the data after dropping the duplicated rows
data.describe()

### __Text Preprocessing__
1. Lowercasing
2. URLs Handling
3. User Mentions Handling
4. English Abbreviations & Slang Handling
5. English Contractions Handling
6. Punctuation & Special Characters Handling
7. Stopwords Handling
8. Emoji/Emoticon Handling via regex
9. Spell Checking
10. Tokenization
11. Lemmatization

Exploratory visualization
    Top 20 most frequent stems (bar chart)
    Word cloud of stem frequencies

In [ ]:
# converting all text to lowercase
data['text'] = data['text'].str.lower()
data.head()

In [ ]:
# removing URLs
data['text'] = data['text'].apply(lambda x: re.sub(r'http\S+', '', x))
data.head()

In [ ]:
# removing user mentions
data['text'] = data['text'].apply(lambda x: re.sub(r'@\S+', '', x))
data.head()

In [ ]:
# changing abbreviations and slang to their standard forms
abbreviation_dict = {
    "bked": "booked",
    "thx": "thanks",
    "plz": "please",
    "sfo": "san francisco airport",
    "lax": "los angeles airport",
    "nyc": "new york city",
    "bos": "boston",
    "las": "las vegas",
    "dal": "dallas",
    "dca": "washington, d.c.",
    "lg": "likely good"
}

def text_std(text):
    words = text.split()
    new_words = []
    for word in words:
        if word in abbreviation_dict:
            word = abbreviation_dict[word]
        new_words.append(word)
    return " ".join(new_words)

data['text'] = data['text'].apply(text_std)
data.head()

In [ ]:
# handling english contractions
english_contractions_dict = {
    "ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have",
    "could've": "could have", "couldn't": "could not", "couldn't've": "could not have",
    "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not",
    "hasn't": "has not", "haven't": "have not", "he'd": "he would", "he'll": "he will",
    "he's": "he is", "how'd": "how did", "how'll": "how will", "how's": "how is",
    "i'd": "i would", "i'll": "i will", "i'm": "i am", "i've": "i have", "isn't": "is not",
    "it'd": "it would", "it'll": "it will", "it's": "it is", "let's": "let us",
    "ma'am": "madam", "might've": "might have", "mightn't": "might not", "must've": "must have",
    "mustn't": "must not", "needn't": "need not", "shan't": "shall not", "she'd": "she would",
    "she'll": "she will", "she's": "she is", "should've": "should have", "shouldn't": "should not",
    "that'd": "that would", "that's": "that is", "there's": "there is", "they'd": "they would",
    "they'll": "they will", "they're": "they are", "they've": "they have", "wasn't": "was not",
    "we'd": "we would", "we're": "we are", "we've": "we have", "weren't": "were not",
    "what'll": "what will", "what're": "what are", "what's": "what is", "what've": "what have",
    "where's": "where is", "who's": "who is", "who've": "who have", "won't": "will not",
    "would've": "would have", "wouldn't": "would not", "you'd": "you would", "you'll": "you will",
    "you're": "you are", "you've": "you have"
}

def text_std(text):
    words = text.split()
    new_words = []
    for word in words:
        if word in english_contractions_dict:
            word = english_contractions_dict[word]
        new_words.append(word)
    return " ".join(new_words)

data['text'] = data['text'].apply(text_std)
data.head()

In [ ]:
# removing punctuation and special characters
data['text'] = data['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
data.head()

In [ ]:
# removing stopwords
english_stopwords = stopwords.words("english")
data['text'] = data['text'].apply(lambda x: " ".join(x for x in x.split() if x.lower() not in english_stopwords))
data.head()

In [ ]:
# dealing with emojis (btw: data has no emojis or emoticons)
from emot.emo_unicode import UNICODE_EMOJI, EMOTICONS_EMO
import re

def convert_emojis(text):
    for emot in UNICODE_EMOJI: # like 😊
        if emot in text:
            print(emot)
            text = text.replace(
                emot,
                UNICODE_EMOJI[emot]
                    .replace(":", "")
                    .replace(",", "")
                    .replace("_", " ")
                ).lower()
            
    for emo in EMOTICONS_EMO: # like :‑)
        if emo in text:
            print(emo)
            text = text.replace(
                emo,
                EMOTICONS_EMO[emo]
                    .replace(":", "")
                    .replace(",", "")
                    .replace("_", " ")
                ).lower()
    return text

data['text'] = data['text'].apply(convert_emojis)
data.head()

In [ ]:
import pandas as pd
from autocorrect import Speller

spell = Speller(lang='en')

all_text = ' '.join(data['text'].astype(str).tolist())
unique_words = set(all_text.lower().split())

# correcting unique words: creating a dict of unique words and their corrected spellings
word_corrections = {word: spell(word) for word in unique_words}

def correct_sentence_with_map(sentence, corrections_map):
    words = sentence.split()
    corrected_words = [corrections_map.get(word.lower(), word) for word in words]

    return ' '.join(corrected_words)

data['text'] = data['text'].astype(str).apply(lambda x: correct_sentence_with_map(x, word_corrections))
data.head()

In [423]:
# tokenizing the text
data['text'] = data['text'].apply(word_tokenize)
data.head()

In [ ]:
# lemmatizing the text
data['text'] = data['text'].apply(Word)
data.head()